<a href="https://colab.research.google.com/github/amitranjan02/Berkeley-433.1/blob/master/Example_1_Load_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Read a CSV file from web.
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data')
print(df[:10])

   vhigh vhigh.1  2 2.1  small   low  unacc
0  vhigh   vhigh  2   2  small   med  unacc
1  vhigh   vhigh  2   2  small  high  unacc
2  vhigh   vhigh  2   2    med   low  unacc
3  vhigh   vhigh  2   2    med   med  unacc
4  vhigh   vhigh  2   2    med  high  unacc
5  vhigh   vhigh  2   2    big   low  unacc
6  vhigh   vhigh  2   2    big   med  unacc
7  vhigh   vhigh  2   2    big  high  unacc
8  vhigh   vhigh  2   4  small   low  unacc
9  vhigh   vhigh  2   4  small   med  unacc


In [0]:
# Download a CSV to local disk, then read from local disk.
import requests
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
r = requests.get(url, allow_redirects=True)
open('car.data', 'wb').write(r.content)
import pandas as pd
df = pd.read_csv('car.data')
print(df[:10])

   vhigh vhigh.1  2 2.1  small   low  unacc
0  vhigh   vhigh  2   2  small   med  unacc
1  vhigh   vhigh  2   2  small  high  unacc
2  vhigh   vhigh  2   2    med   low  unacc
3  vhigh   vhigh  2   2    med   med  unacc
4  vhigh   vhigh  2   2    med  high  unacc
5  vhigh   vhigh  2   2    big   low  unacc
6  vhigh   vhigh  2   2    big   med  unacc
7  vhigh   vhigh  2   2    big  high  unacc
8  vhigh   vhigh  2   4  small   low  unacc
9  vhigh   vhigh  2   4  small   med  unacc


In [0]:
# Read to and write from remote file system (AWS S3)
import boto3, os

# Upload to AWS
s3 = boto3.resource('s3',
                    aws_access_key_id='XXXXXXX',
                    aws_secret_access_key='XXXXXXX')
# s3.create_bucket(Bucket='datascienceclass-datasets')
s3.meta.client.upload_file('car.data', 'datascienceclass-datasets', 'car.data')
# Read the file.
for obj in s3.Bucket(name='datascienceclass-datasets').objects.all():
    print(os.path.join(obj.bucket_name, obj.key))
    import pandas as pd
    from io import StringIO
    # Read the file content, decode it and parse it.
    content_bytes = obj.get()['Body'].read()
    print('content type: ', type(content_bytes))
    content = content_bytes.decode()
    print('decoded content type: ', type(content))
    print(pd.read_csv(StringIO(str(content)))[:10])

datascienceclass-datasets\car.data
content type:  <class 'bytes'>
decoded content type:  <class 'str'>
   vhigh vhigh.1  2 2.1  small   low  unacc
0  vhigh   vhigh  2   2  small   med  unacc
1  vhigh   vhigh  2   2  small  high  unacc
2  vhigh   vhigh  2   2    med   low  unacc
3  vhigh   vhigh  2   2    med   med  unacc
4  vhigh   vhigh  2   2    med  high  unacc
5  vhigh   vhigh  2   2    big   low  unacc
6  vhigh   vhigh  2   2    big   med  unacc
7  vhigh   vhigh  2   2    big  high  unacc
8  vhigh   vhigh  2   4  small   low  unacc
9  vhigh   vhigh  2   4  small   med  unacc


In [0]:
# Write and then read from database.
import boto3
dynamodb = boto3.resource('dynamodb',
                          region_name='us-west-1',
                          aws_access_key_id='XXXXXX',
                          aws_secret_access_key='XXX')
try:
    table = dynamodb.create_table(
        TableName='uci_car',
        KeySchema=[
            {
                'AttributeName': 'id',
                'KeyType': 'HASH'
            },
        ], 
        AttributeDefinitions=[
            {
                'AttributeName': 'id', 
                'AttributeType': 'S'
            },  
        ], 
        ProvisionedThroughput={
            'ReadCapacityUnits': 1, 
            'WriteCapacityUnits': 1
        }
    )
except:
    pass
table = dynamodb.Table('uci_car')

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data')
for idx, item in enumerate(df.values[:10]):
    table.put_item(
       Item={
           'id': str(idx),
           # TODO: add other features here..
           'buying': str(item[0]),
           # 'maint': str(item[1]),
           'label': str(item[-1])
        }
    )

response = table.get_item(
   Key={
        'id': '0',
    }
)
print(response)

table.delete()

{'Item': {'id': '0', 'buying': 'vhigh', 'label': 'unacc'}, 'ResponseMetadata': {'RequestId': 'KGVIQD7FQU369QCCUSC09RKAIBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sun, 20 Jan 2019 20:57:49 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '70', 'connection': 'keep-alive', 'x-amzn-requestid': 'KGVIQD7FQU369QCCUSC09RKAIBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2800687844'}, 'RetryAttempts': 0}}


{'TableDescription': {'TableName': 'uci_car',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 1,
   'WriteCapacityUnits': 1},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-west-1:106493381494:table/uci_car',
  'TableId': 'aa3fb573-aff1-4516-b8fd-2fcd3f85901f'},
 'ResponseMetadata': {'RequestId': 'LQ92FEPPO8K73D3L9NF8DM32D7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 20 Jan 2019 20:57:49 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '316',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'LQ92FEPPO8K73D3L9NF8DM32D7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1580782858'},
  'RetryAttempts': 0}}

In [0]:
# Process JSON format. Examples from: https://docs.python-guide.org/scenarios/json/
import json
json_string = '{"first_name": "Guido", "last_name":"Rossum"}'
parsed_json = json.loads(json_string)
print(type(parsed_json))
print(parsed_json)
print(json.dumps({'first_name': 'Guido', 'last_name': 'Rossum'}))

# rich representation
print(json.dumps({'image': 'url', 'text': ['string1', 'string2']}))

<class 'dict'>
{'first_name': 'Guido', 'last_name': 'Rossum'}
{"first_name": "Guido", "last_name": "Rossum"}
{"image": "url", "text": ["string1", "string2"]}
